
New agricultural value pseudocode:
	1. Mask landcover to just cropland
	2. Reclassify cropland to 1
	3. Mask cropland to TCD_ADM0
	4. Raster to polygon with Simplify Features checked. (If vertices estimate is already known, can use the maximum vertices limit and skip Dice tool)
	5. Calculate new field $area
	6. Calculate new field VERTEXCT as: !shape!.pointcount and sort by max. Spot check among the largest features to decide what number of vertices gives an appropriate maximum crop area. Can delete this field once this value is known.
	7. Dice polygon. Vertices limit = the vertex count determined in the previous step.
	8. Centroids of polygon
	9. Spatial join crop points within agro9km polygons
	10. Group by agro_ID: proportion (%) of crop area per feature (feature's area / sum of all crop areas agro_id)
Agroval_pt = prop_area * agro_val9km


1.	SPAM_ply = SPAM raster to polygons
2.	crop = Mask landcover to just cropland
3.	crop = Reclassify cropland to 1
4.	crop = Mask cropland to TCD_ADM0
5.	crop_ply = Raster to polygon with Simplify Features checked, max vertices = 100
6.	crop_ply = calculate new field $area
7.	crop_pt = Centroids of crop_ply
8.	agro_crop = Spatial join crop points within agro9km polygons
9.	agro_crop = Group by agro_ID: proportion (%) of crop area per feature (feature's area / sum of all crop areas agro_id)
10.	agro_crop$agroval = prop_area * agro_val9km

In [1]:
import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [2]:
def ListFromRange(r1, r2):
    return [item for item in range(r1, r2+1)]

In [3]:
# From Stack Exchange @RutgerH
# https://gis.stackexchange.com/questions/163685/reclassify-a-raster-value-to-9999-and-set-it-to-the-nodata-value-using-python-a
def readRaster(filename):
    filehandle = gdal.Open(filename)
    band1 = filehandle.GetRasterBand(1)
    geotransform = filehandle.GetGeoTransform()
    geoproj = filehandle.GetProjection()
    Z = band1.ReadAsArray()
    xsize = filehandle.RasterXSize
    ysize = filehandle.RasterYSize
    return xsize,ysize,geotransform,geoproj,Z

In [108]:
# Default arguments can be changed here, or can be specified below when running the functions.
def writeRaster(filename,geotransform,geoprojection,data, NoDataVal=0):
    (x,y) = data.shape
    Dformat = "GTiff"
    driver = gdal.GetDriverByName(Dformat)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_datatype = gdal.GDT_Float32
    dst_ds = driver.Create(filename,y,x,1,dst_datatype)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds.GetRasterBand(1).SetNoDataValue(NoDataVal)
    return 1
    dst_ds = None

In [5]:
# Based on Stack Exchange @Kurt Schwehr:
# https://stackoverflow.com/questions/10454316/how-to-project-and-resample-a-grid-to-match-another-grid-with-gdal-python
def resampleRaster(InRaster_Path, MatchRaster_Path, OutFile_Path):
    print('Loading for %s. %s' % (InRaster_Path, time.ctime()))
    
    RasterObject = gdal.Open(InRaster_Path)
    In_proj = RasterObject.GetProjection()
    [Match_x, Match_y, Match_geo, Match_proj, Match_Z] = readRaster(MatchRaster_Path)
    print('---Specs to match to: \n', 
      Match_proj, '\n', Match_geo, '\n', Match_x, '\n', Match_y, '\n')
        
    OutFile = gdal.GetDriverByName('GTiff').Create(OutFile_Path, Match_x, Match_y, 1, gdalconst.GDT_UInt32)
    OutFile.SetGeoTransform(Match_geo)
    OutFile.SetProjection(Match_proj)
    print('---Created raster file for upsampled version. %s' % time.ctime())
    
    gdal.ReprojectImage(RasterObject, OutFile, In_proj, Match_proj, gdal.GRA_Bilinear)
    print('---Resampled flood values onto an empty raster matching the dimensions of the buildup layer. %s \n\n' % time.ctime())
    
    OutFile.GetRasterBand(1).SetNoDataValue(0)
    
    RasterObject = Outfile = None
    return 1

In [6]:
def calcShell(A, B, OutFile, Calculation, OutType = '', 
              C=None, D=None, E=None, F=None, G=None):
    """Raster math using gdal_calc.py.

    The OSgeo package for Python API does not make raster calculations
    easy outside of the shell. This function plugs up to 6 raster files
    into a string which subprocess.call() then commits to the terminal.

        A : str
            File path to the first raster for the calculation.
        B : str
            File path to the second raster for the calculation.
        OutFile : str
            File path where to store the raster generated from the calculation.
        Calculation : str
            Algebra that uses A and B to create a new raster. Use double quotes.
    """
    print('Running for %s. %s' % (A, time.ctime()))
    cmd = 'gdal_calc.py -A ' + A + ' -B ' + B 
    if C is not None:
        cmd = cmd + ' -C' + C 
    if D is not None:
        cmd = cmd + ' -D' + D
    if E is not None:
        cmd = cmd + ' -E' + E
    if F is not None:
        cmd = cmd + ' -F' + F
    if G is not None:
        cmd = cmd + ' -G' + G
    cmd = cmd + OutType + ' --outfile=' + OutFile + ' --overwrite --calc=' + Calculation
    subprocess.call(cmd, shell=True)
    cmd = A = B = C = D = E = F = G = None
    print('Ran in shell. See OutFile folder to inspect results. %s' % time.ctime())

In [7]:
def mosaicShell(A, B, OutFile, Band = 1, 
                  C=None, D=None, E=None, F=None, G=None):
    print('Running for %s. %s' % (A, time.ctime()))
    
    StringFiles = ' '.join([A,B])
    
    for RasterName in [C,D,E,F,G]:
        if RasterName is not None:
            StringFiles = ' '.join([StringFiles, RasterName])
        else:
            pass
        
    cmd = 'gdal_merge.py -o ' + OutFile + ' -of gtiff ' + StringFiles
    
    subprocess.call(cmd, shell=True)
    print('Ran in shell. See OutFile folder to inspect results. %s' % time.ctime())

In [8]:
def RasterToShapefile(InRasterPath, OutFilePath = 'RastToShp.shp', Band=1, 
                      OutName='RastToShp', VariableName='value', Driver = 'ESRI Shapefile'):
    """Raster tiff to vector polygon shapefile.
    
    """
    Raster = gdal.Open(InRasterPath)
    RasterBand = Raster.GetRasterBand(Band)
    
    OutDriver = ogr.GetDriverByName(Driver)
    InProj = Raster.GetProjectionRef()
    SpatRef = osr.SpatialReference()
    SpatRef.ImportFromWkt(InProj)
    print(InProj, '\n\n', SpatRef)
    
    OutFile = OutDriver.CreateDataSource(OutFilePath)
    OutLayer = OutFile.CreateLayer(OutName, srs = SpatRef, geom_type = ogr.wkbPolygon)
    OutField = ogr.FieldDefn(VariableName, ogr.OFTInteger)
    OutLayer.CreateField(OutField)
    OutField = OutLayer.GetLayerDefn().GetFieldIndex(VariableName)
    print('\n', OutFile, '\n', OutLayer, '\n', OutField)
    
    print('Vectorizing. Input: %s. %s' % (InRasterPath, time.ctime()))
    gdal.Polygonize(RasterBand, None, OutLayer, 0, [], callback=None)
    print('Completed polygons. Stored as: %s. %s' % (OutFilePath, time.ctime()))

    del Raster, RasterBand, OutFile, OutLayer

In [9]:
def rioStats(InRasterPath, Band = 1):
    out = rasterio.open(InRasterPath)
    stats = []
    band = out.read(Band)
    stats.append({
        'raster': out.name,
        'bands': out.count,
        'data type': out.dtypes,
        'no data value': out.nodatavals,
        'width': out.width,
        'height': out.height,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})
    print("\n", stats)
    
    out = band = None

In [10]:
def SimplifyFileNames(FilesList, folder):
    i = 1
    while i <= len(FilesList):
        for filename in FilesList:
            dst = ''.join(['Crop', str(i), '.tif'])
            src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
            dst =f"{folder}/{dst}"
            os.rename(src, dst)
            
            i += 1

In [11]:
def MaskByZone(MaskPath, SourceFolder, DestFolder, MaskLayerName = None, dstSRS = 'ESRI:102022'):
    """
    Reduces the size of a raster's valid data cells to vector areas of interest.
    This is useful if the raster data needs to be vectorized later to save space.
    
    The script prepares the vector zones as a list of geometries in the desired
    spatial reference system, then warps each raster in the specified source
    folder to the same SRS. Masking in rasterio then reclassifies any raster cells
    falling outside of a mask polygon as NoData.
    """
    
    ProjSRS = osr.SpatialReference()
    ProjSRS.SetFromUserInput(dstSRS)
    ProjWarp = gdal.WarpOptions(dstSRS = dstSRS)
    
    SourceFiles = []
    SourceFiles = SourceFiles + [i for i in os.listdir(''.join([SourceFolder, r'/'])) if i.endswith('tif')]
    print(SourceFiles)
    
    
    ### 1. ASSIGN SPATIAL REFERENCE SYSTEM OF VECTOR MASK AND LOAD GEOMETRIES
    Vector = gpd.read_file(filename=MaskPath, layer=MaskLayerName)
    if Vector.crs != dstSRS:
        if MaskLayerName == None:
            MaskPath = MaskPath + '_temp'
        else:
            MaskLayerName = MaskLayerName + '_temp'
        Vector.to_crs(dstSRS).to_file(filename=MaskPath, layer=MaskLayerName)
    Vector = None # We're reloading the geometries with fiona
    
    with fiona.open(MaskPath, mode="r", layer=MaskLayerName) as Vector:
        MaskGeom = [feature["geometry"] for feature in Vector] # Identify the bounding areas of the mask.
    
    
    ### 2. PREPARE DESTINATION FILES
    for FileName in SourceFiles:
        InputRasterPath = os.path.join(ProjectFolder, SourceFolder, FileName)
        
        TempOutputName = 'Temp_' + FileName
        TempOutputPath = os.path.join(ProjectFolder, DestFolder, TempOutputName)
        FinalOutputName = 'Msk_' + FileName
        FinalOutputPath = os.path.join(ProjectFolder, DestFolder, FinalOutputName)

    ### 3. ASSIGN SPATIAL REFERENCE SYSTEM OF RASTER(S)
        InputRasterObject = gdal.Open(InputRasterPath)
        SourceSRS = osr.SpatialReference(wkt=InputRasterObject.GetProjection())
        print('Source projection: ', SourceSRS.GetAttrValue('projcs'))
        print('Destination projection: ', ProjSRS.GetAttrValue('projcs'))

        if SourceSRS.GetAttrValue('projcs') != ProjSRS.GetAttrValue('projcs'):
            Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                         InputRasterObject, # Which raster to warp
                         format='GTiff', 
                         options=ProjWarp) # Reproject to Africa Albers Equal Area Conic
            print('Finished gdal.Warp() for %s. %s \n' % (FileName, time.ctime()))

            Warp = None # Close the files
        else:
            pass
        InputRasterObject = None
        
    ### 4. RECLASSIFY AS NODATA IF OUTSIDE OF ZONE.
        if exists(TempOutputPath):
            NewInputPath = TempOutputPath 
            print("We warped the data, so we'll use that file for next step.")
        else:
            NewInputPath = InputRasterPath 
            print("We skipped the warp, so we continue to use the source file.")

        with rasterio.open(NewInputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for %s. %s \n' % (FileName, time.ctime()))

        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})

        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
        InputRasterObject = None

        if exists(TempOutputPath):
            try:  # Finally, remove the intermediate file from disk
                os.remove(TempOutputPath)
            except OSError:
                pass
            print('Removed intermediate file. %s \n' % time.ctime())
        else:
            pass


    print('\n \n Finished all items in list. %s' % time.ctime())

In [12]:
ProjectFolder = os.getcwd()
print(ProjectFolder)

Q:\GIS\povertyequity\mauritania\Agro_MRT


## 1. Combine SPAM crops' values

#### Select only rainfed, low input crop values:

In [16]:
SPAMcrops = []
SPAMcrops = SPAMcrops + [i for i in os.listdir(''.join([ProjectFolder, r'/SPAM/SourceFiles/'])) if i.endswith('L.tif')]
print(SPAMcrops, '\n\nNumber of crops: ', len(SPAMcrops))

['spam2017V2r1_SSA_V_ACOF_L.tif', 'spam2017V2r1_SSA_V_BANA_L.tif', 'spam2017V2r1_SSA_V_BARL_L.tif', 'spam2017V2r1_SSA_V_BEAN_L.tif', 'spam2017V2r1_SSA_V_CASS_L.tif', 'spam2017V2r1_SSA_V_CHIC_L.tif', 'spam2017V2r1_SSA_V_CNUT_L.tif', 'spam2017V2r1_SSA_V_COCO_L.tif', 'spam2017V2r1_SSA_V_COTT_L.tif', 'spam2017V2r1_SSA_V_COWP_L.tif', 'spam2017V2r1_SSA_V_GROU_L.tif', 'spam2017V2r1_SSA_V_LENT_L.tif', 'spam2017V2r1_SSA_V_MAIZ_L.tif', 'spam2017V2r1_SSA_V_OCER_L.tif', 'spam2017V2r1_SSA_V_OFIB_L.tif', 'spam2017V2r1_SSA_V_OILP_L.tif', 'spam2017V2r1_SSA_V_OOIL_L.tif', 'spam2017V2r1_SSA_V_OPUL_L.tif', 'spam2017V2r1_SSA_V_ORTS_L.tif', 'spam2017V2r1_SSA_V_PIGE_L.tif', 'spam2017V2r1_SSA_V_PLNT_L.tif', 'spam2017V2r1_SSA_V_PMIL_L.tif', 'spam2017V2r1_SSA_V_POTA_L.tif', 'spam2017V2r1_SSA_V_RAPE_L.tif', 'spam2017V2r1_SSA_V_RCOF_L.tif', 'spam2017V2r1_SSA_V_REST_L.tif', 'spam2017V2r1_SSA_V_RICE_L.tif', 'spam2017V2r1_SSA_V_SESA_L.tif', 'spam2017V2r1_SSA_V_SMIL_L.tif', 'spam2017V2r1_SSA_V_SORG_L.tif', 'spam2017

#### Confirm all rasters are aligned

In [21]:
for Crop in SPAMcrops:
    RasterPath = os.path.join(ProjectFolder, 'SPAM/SourceFiles', Crop)
    [Match_x, Match_y, Match_geo, Match_proj, Match_Z] = readRaster(RasterPath)
    print(Crop, ': \n', 
      Match_proj, '\n', Match_geo, '\n', Match_x, '\n', Match_y, '\n')

spam2017V2r1_SSA_V_ACOF_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] 
 (-180.0, 0.083333, 0.0, 90.0, 0.0, -0.083333) 
 4320 
 2160 

spam2017V2r1_SSA_V_BANA_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] 
 (-180.0, 0.083333, 0.0, 90.0, 0.0, -0.083333) 
 4320 
 2160 

spam2017V2r1_SSA_V_BARL_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","

spam2017V2r1_SSA_V_POTA_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] 
 (-180.0, 0.083333, 0.0, 90.0, 0.0, -0.083333) 
 4320 
 2160 

spam2017V2r1_SSA_V_RAPE_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] 
 (-180.0, 0.083333, 0.0, 90.0, 0.0, -0.083333) 
 4320 
 2160 

spam2017V2r1_SSA_V_RCOF_L.tif : 
 GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","

In [19]:
for Crop in SPAMcrops:
    RasterPath = os.path.join(ProjectFolder, 'SPAM/SourceFiles', Crop)
    rioStats(RasterPath)


 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_ACOF_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 26.176563, 'median': -1.0, 'max': 2427903.5}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_BANA_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 118.348076, 'median': -1.0, 'max': 20895364.0}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_BARL_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 4.9961658, 'median': -1.0, 'max': 401078.0}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_BEAN_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 159.24501, 'median': 


 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_SWPO_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 94.754364, 'median': -1.0, 'max': 3342256.8}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_TEAS_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 39.98859, 'median': -1.0, 'max': 7224164.0}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_TEMF_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 115.583115, 'median': -1.0, 'max': 14821220.0}]

 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/spam2017V2r1_SSA_V_TOBA_L.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (-1.0,), 'width': 4320, 'height': 2160, 'min': -1.0, 'mean': 36.577988, 'median': 

#### Rename value of production rasters for ease of use in gdal_calc

In [52]:
SimplifyFileNames(SPAMcrops, os.path.join(ProjectFolder, 'SPAM/SourceFiles'))

In [54]:
SPAMcrops = []
SPAMcrops = SPAMcrops + [i for i in os.listdir(''.join([ProjectFolder, r'/SPAM/SourceFiles/'])) if i.startswith('Crop')]
print(SPAMcrops, '\n\nNumber of crops: ', len(SPAMcrops))

['Crop1.tif', 'Crop10.tif', 'Crop11.tif', 'Crop12.tif', 'Crop13.tif', 'Crop14.tif', 'Crop15.tif', 'Crop16.tif', 'Crop17.tif', 'Crop18.tif', 'Crop19.tif', 'Crop2.tif', 'Crop20.tif', 'Crop21.tif', 'Crop22.tif', 'Crop23.tif', 'Crop24.tif', 'Crop25.tif', 'Crop26.tif', 'Crop27.tif', 'Crop28.tif', 'Crop29.tif', 'Crop3.tif', 'Crop30.tif', 'Crop31.tif', 'Crop32.tif', 'Crop33.tif', 'Crop34.tif', 'Crop35.tif', 'Crop36.tif', 'Crop37.tif', 'Crop38.tif', 'Crop39.tif', 'Crop4.tif', 'Crop40.tif', 'Crop41.tif', 'Crop42.tif', 'Crop5.tif', 'Crop6.tif', 'Crop7.tif', 'Crop8.tif', 'Crop9.tif'] 

Number of crops:  42


#### Run gdal_calc in batches to avoid overload.

In [59]:
i = 1
Calc = 'A+B+C+D+E+F+G'

while i < 43:
    C1 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i), '.tif']))
    C2 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+1), '.tif']))
    C3 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+2), '.tif']))
    C4 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+3), '.tif']))
    C5 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+4), '.tif']))
    C6 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+5), '.tif']))
    C7 = os.path.join(ProjectFolder, 'SPAM/SourceFiles', ''.join(['Crop', str(i+6), '.tif']))
    OutName = ''.join(['Subset', str(i), 'to', str(i+6), '.tif'])
    calcShell(A=C1, B=C2, C=C3, D=C4, E=C5, F=C6, G=C7, 
              OutFile = os.path.join(ProjectFolder, 'SPAM', OutName), 
              Calculation = Calc)
    i += 7

Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop1.tif. Fri Feb 24 19:12:47 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 19:12:54 2023
Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop8.tif. Fri Feb 24 19:12:54 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 19:13:01 2023
Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop15.tif. Fri Feb 24 19:13:01 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 19:13:07 2023
Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop22.tif. Fri Feb 24 19:13:07 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 19:13:14 2023
Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop29.tif. Fri Feb 24 19:13:14 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 19:13:21 2023
Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Crop36.tif. Fri Feb 24 19:13:21 2023
Ran in shell. See OutFile folder to inspec

In [83]:
SPAMcrops = []
SPAMcrops = SPAMcrops + [i for i in os.listdir(''.join([ProjectFolder, r'/SPAM/'])) if i.startswith('Subset')]
SPAMcrops

['Subset15to21.tif',
 'Subset1to7.tif',
 'Subset22to28.tif',
 'Subset29to35.tif',
 'Subset36to42.tif',
 'Subset8to14.tif']

#### Add the batches together for final total

In [84]:
Calc = 'A+B+C+D+E+F'

C1 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[0])
C2 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[1])
C3 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[2])
C4 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[3])
C5 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[4])
C6 = os.path.join(ProjectFolder, 'SPAM', SPAMcrops[5])
OutName = 'SPAM_ProdVal_2017.tif'
calcShell(A=C1, B=C2, C=C3, D=C4, E=C5, F=C6, 
          OutFile = os.path.join(ProjectFolder, 'SPAM', OutName), 
          Calculation = Calc)

Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\SPAM\Subset15to21.tif. Sat Feb 25 12:05:13 2023
Ran in shell. See OutFile folder to inspect results. Sat Feb 25 12:05:20 2023


#### I get into a lot of issues if I'm not in WGS84 for some of these functions. Quick warp.

In [101]:
ProjSRS = osr.SpatialReference()
ProjSRS.SetFromUserInput('EPSG:4326')
ProjWarp = gdal.WarpOptions(dstSRS = 'EPSG:4326')

InputRasterObject = gdal.Open('SPAM/SPAM_ProdVal_2017.tif')
OutputRasterPath = 'SPAM/SPAM_ProdVal_2017_WGS84.tif'
SourceSRS = osr.SpatialReference(wkt=InputRasterObject.GetProjection())

Warp = gdal.Warp(OutputRasterPath, # Where to store the warped raster
             InputRasterObject, # Which raster to warp
             format='GTiff', 
             options=ProjWarp) 
print('Finished gdal.Warp(). %s \n' % (time.ctime()))

Warp = None # Close the files

InputRasterObject = OutputRasterPath = None

Finished gdal.Warp(). Sat Feb 25 12:21:00 2023 



#### Before masking, make sure the NoData value is what we want.

In [102]:
RasterPath = os.path.join(ProjectFolder, 'SPAM', 'SPAM_ProdVal_2017_WGS84.tif')
rioStats(RasterPath)


 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/SPAM_ProdVal_2017_WGS84.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (3.4028234663852886e+38,), 'width': 4320, 'height': 2160, 'min': nan, 'mean': nan, 'median': nan, 'max': nan}]


In [103]:
[xsize,ysize,geotransform,geoproj,Z] = readRaster('SPAM/SPAM_ProdVal_2017_WGS84.tif')
print(xsize, ysize, geotransform, geoproj, Z)

4320 2160 (-180.0, 0.08333333333333333, 0.0, 90.0, 0.0, -0.083333) GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [104]:
print('Nan values: ', np.count_nonzero(np.isnan(Z)), '\n')
print('Numeric values: ', np.count_nonzero(~np.isnan(Z)))

Nan values:  9186874 

Numeric values:  144326


In [105]:
N = np.nan_to_num(Z)
print(N)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [106]:
print(N[N>0][:20]) # Print first 20 values that are not 0.

[  2268.4     8687.2   198659.5     1241.1     1883.1      642.
   1027.1     3222.2     2311.2     1069.9    18886.5    17877.4
  64844.5    17877.4     1284.      1284.      2439.5     1284.
    747.6    13429.101]


In [95]:
N = N.astype('uint32')

In [109]:
writeRaster('SPAM/TotVal/SPAM_ProdVal_2017_v2.tif',geotransform,geoproj,N, NoDataVal=0) 

1

#### Crop to country

In [68]:
MaskByZone(MaskPath='ADM/MRT_ADM0.shp', SourceFolder='SPAM/TotVal', DestFolder='SPAM', dstSRS = 'ESRI:102022')

['SPAM_ProdVal_2017.tif']
Source projection:  None
Destination projection:  None
We skipped the warp, so we continue to use the source file.
Finished rasterio.mask.mask() for SPAM_ProdVal_2017.tif. Sat Feb 25 11:39:12 2023 

Written to file. Sat Feb 25 11:39:12 2023 


 
 Finished all items in list. Sat Feb 25 11:39:12 2023


## 2. Convert SPAM agricultural value to polygon

In [69]:
RasterPath = os.path.join(ProjectFolder, 'SPAM', 'Msk_SPAM_ProdVal_2017.tif')
rioStats(RasterPath)


 [{'raster': 'Q:/GIS/povertyequity/mauritania/Agro_MRT/SPAM/Msk_SPAM_ProdVal_2017.tif', 'bands': 1, 'data type': ('float32',), 'no data value': (3.4028234663852886e+38,), 'width': 148, 'height': 152, 'min': nan, 'mean': nan, 'median': nan, 'max': nan}]


C:\Users\grace\miniconda3\envs\geo\lib\site-packages\numpy\core\_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [61]:
[xsize,ysize,geotransform,geoproj,Z] = readRaster('SPAM/Msk_SPAM_ProdVal_2017.tif')
print(xsize, ysize, geotransform, geoproj, Z)

152 164 (-4407004.614664944, 8471.910742119699, 0.0, 3080913.283621446, 0.0, -8471.910742119699) PROJCS["Africa_Albers_Equal_Area_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",25],PARAMETER["standard_parallel_1",20],PARAMETER["standard_parallel_2",-23],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] [[3.4028235e+38 3.4028235e+38 3.4028235e+38 ... 3.4028235e+38
  3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 ... 3.4028235e+38
  3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 ... 3.4028235e+38
  3.4028235e+38 3.4028235e+38]
 ...
 [3.4028235e+38 3.402

In [62]:
print(Z.dtype)

float32


In [63]:
print(Z[Z>0])

[3.4028235e+38 3.4028235e+38 3.4028235e+38 ... 3.4028235e+38 3.4028235e+38
 3.4028235e+38]


In [58]:
Z = Z.astype('uint32')
print(Z.dtype)
print(Z)

uint32
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [59]:
print(Z[Z>0])

[  2268   8687   8687   1241   1883    642   1027   2311   1069   1284
   1284   2439   2439   1284   2311   1284   1284   2576   2311   3353
   2576   2311   1711   2311   1284   2439    642   2439   2576   2439
   6641   8870   2311   1284   2439   2439   2896   1284   2667   6504
  13300   2311   2311   2311   2576   3754   2311   1284   1284   1284
   4495   2439   2454   2745   2311   2684   2592   4769   2439   1069
   2896   2896   2804  14452   2439   2643   5068   2516   2454   2439
   1711   3444   2576   2868   4618   2439   1284   1711   1711   1284
   5363   2896   6139   2439   2439   2439    856    856   1284   5363
   2896   6139   2439   3982   2311   1069   1711   4074   3982   2140
   2987   1284   2353    214   4819   2576   5043   1069   2353   3486
   4358   4632   2896   1284   2353   1284   2353   6413   3845   2140
   1284   1284   9291  17092   2439    385   2576   2439   1284   2439
   6322   2439  16817   1284   2759    770   2353   3855   3444   2439
   335

In [43]:
# N = np.where(Z<0, 0, Z)
# N

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [54]:
# N = np.multiply(N, 100, where=(N>0))
# #N = N.astype('uint64')

In [55]:
# N[N>0]

array([ 623964176,        600,  559020848, ..., 2139095039, 2139095039,
       2139095039], dtype=uint32)

In [60]:
writeRaster('SPAM/ProdVal_2017_v2.tif',geotransform,geoproj,Z, NoDataVal=0) 

1

## 1. Mosaic UMD GLAD land cover

In [21]:
GLADpaths = []

for i in os.listdir(''.join([ProjectFolder, r'/GLAD/'])):
    k = os.path.join(ProjectFolder, 'GLAD', i)
    GLADpaths = GLADpaths + [k]
    
GLADpaths

['Q:\\GIS\\povertyequity\\mauritania\\Agro_MRT\\GLAD\\20N_010W.tif',
 'Q:\\GIS\\povertyequity\\mauritania\\Agro_MRT\\GLAD\\20N_020W.tif',
 'Q:\\GIS\\povertyequity\\mauritania\\Agro_MRT\\GLAD\\30N_010W.tif',
 'Q:\\GIS\\povertyequity\\mauritania\\Agro_MRT\\GLAD\\30N_020W.tif']

In [23]:
mosaicShell(A=GLADpaths[0], B=GLADpaths[1], C=GLADpaths[2], D=GLADpaths[3],
             OutFile = os.path.join(ProjectFolder, 'GLAD', 'LC_strata_mosaic.tif'))

Running for Q:\GIS\povertyequity\mauritania\Agro_MRT\GLAD\20N_010W.tif. Fri Feb 24 20:30:32 2023
Ran in shell. See OutFile folder to inspect results. Fri Feb 24 20:36:49 2023


In [30]:
for i in GLADpaths:
    try:  
        os.remove(os.path.join(i))
        print('Removed intermediate file: %s. %s \n' % (i, time.ctime()))
    except OSError:
        print('Skipped due to error. (Was the file already deleted?). %s. %s \n' % (i, time.ctime()))

Skipped due to error. (Was the file already deleted?). Q:\GIS\povertyequity\mauritania\Agro_MRT\GLAD\20N_010W.tif. Fri Feb 24 20:58:02 2023 

Skipped due to error. (Was the file already deleted?). Q:\GIS\povertyequity\mauritania\Agro_MRT\GLAD\20N_020W.tif. Fri Feb 24 20:58:02 2023 

Skipped due to error. (Was the file already deleted?). Q:\GIS\povertyequity\mauritania\Agro_MRT\GLAD\30N_010W.tif. Fri Feb 24 20:58:02 2023 

Skipped due to error. (Was the file already deleted?). Q:\GIS\povertyequity\mauritania\Agro_MRT\GLAD\30N_020W.tif. Fri Feb 24 20:58:02 2023 



## 3. Mask and reclassify land cover to country's cropland

#### Use the country shapefile to define the limits of the land cover data. Both datasets will be reprojected by the function.

In [37]:
MaskByZone(MaskPath='ADM/MRT_ADM0.shp', SourceFolder='GLAD', DestFolder='GLAD', dstSRS = 'ESRI:102022')

['LC_strata_mosaic.tif', 'Temp_LC_strata_mosaic.tif']
Source projection:  None
Destination projection:  Africa_Albers_Equal_Area_Conic
Finished gdal.Warp() for LC_strata_mosaic.tif. Fri Feb 24 21:13:45 2023 

We warped the data, so we'll use that file for next step.
Finished rasterio.mask.mask() for LC_strata_mosaic.tif. Fri Feb 24 21:14:50 2023 

Written to file. Fri Feb 24 21:15:05 2023 

Removed intermediate file. Fri Feb 24 21:15:05 2023 

Source projection:  Africa_Albers_Equal_Area_Conic
Destination projection:  Africa_Albers_Equal_Area_Conic
We skipped the warp, so we continue to use the source file.
Finished rasterio.mask.mask() for Temp_LC_strata_mosaic.tif. Fri Feb 24 21:15:26 2023 

Written to file. Fri Feb 24 21:15:37 2023 


 
 Finished all items in list. Fri Feb 24 21:15:37 2023


#### Read in the new raster and reclassify so that crop cells = 1 and all else is NoData (0).

In [44]:
[xsize,ysize,geotransform,geoproj,Z] = readRaster('GLAD/Msk_LC_strata_mosaic.tif')
print(xsize, ysize, geotransform, geoproj, Z)

Z[Z<17] = 0
Z[Z>17] = 0

writeRaster('GLAD/LC_CroplandOnly.tif',geotransform,geoproj,Z, NoDataVal=0) 

47569 51473 (-4405175.985618629, 26.858620772433884, 0.0, 3076733.4404738816, 0.0, -26.858620772433884) PROJCS["Africa_Albers_Equal_Area_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",25],PARAMETER["standard_parallel_1",20],PARAMETER["standard_parallel_2",-23],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


1

In [45]:
# It wasn't writing correctly when I tried to include this final reclassification step in the same code block as previous.
[xsize,ysize,geotransform,geoproj,Z] = readRaster('GLAD/LC_CroplandOnly.tif')
print(xsize, ysize, geotransform, geoproj, Z)

Z[Z==17] = 1

writeRaster('GLAD/Cropland.tif',geotransform,geoproj,Z, NoDataVal=0) 

47569 51473 (-4405175.985618629, 26.858620772433884, 0.0, 3076733.4404738816, 0.0, -26.858620772433884) PROJCS["Africa_Albers_Equal_Area_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",25],PARAMETER["standard_parallel_1",20],PARAMETER["standard_parallel_2",-23],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


1

## 4. Create simplified vector features of cropland

## 5. Join datasets

## 6. Calculate value per point